# About the notebook

In [2]:
### import libraries
suppressMessages(suppressWarnings(library(tidyverse)))
suppressMessages(suppressWarnings(library(MPSK)))
suppressMessages(suppressWarnings(library(tsne)))

suppressMessages(suppressWarnings(library(flowCore)))
suppressMessages(suppressWarnings(library(flowStats)))
suppressMessages(suppressWarnings(library(flowViz)))
suppressMessages(suppressWarnings(library(flowMatch)))
suppressMessages(suppressWarnings(library(flowMap)))

suppressMessages(suppressWarnings(library(igraph)))
suppressMessages(suppressWarnings(library(pheatmap)))
suppressMessages(suppressWarnings(library(RColorBrewer)))
suppressMessages(suppressWarnings(library(gplots)))

### set directories
dat_dir = "/data/clintko/SMPK"

# Import data

check the file name

In [11]:
tmp = system(paste("ls -1", dat_dir), intern = TRUE)
for(x in tmp){print(x)}

[1] "C.txt"
[1] "ep8cs_dat_gaussnorm.txt"
[1] "ep8cs_dat_mpsk_cal.txt"
[1] "ep8cs_dat_mpsk_raw.txt"
[1] "ep8cs_mpsk_cal.RDS"
[1] "ep8cs_mpsk_chainSummary.RDS"
[1] "ep8cs_mpsk_raw.RDS"
[1] "ep8cs_mpsk_relab.RDS"
[1] "ep8cs_tsne_gaussnorm.txt"
[1] "ep8cs_tsne_idx.txt"
[1] "ep8cs_tsne_mpsk_cal.txt"
[1] "ep8cs_tsne_raw.txt"
[1] "ep8cs_tsne_test.txt"
[1] "Y_raw.txt"


import the file

In [19]:
### values
Y = read_delim(file.path(dat_dir, "Y_raw.txt"), delim = "\t", col_names = FALSE)
Y = as.matrix(Y)

### label
C = read_delim(file.path(dat_dir, "C.txt"),     delim = "\t", col_names = FALSE)
C = as.matrix(C)

Parsed with column specification:
cols(
  X1 = col_double(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_double(),
  X5 = col_double(),
  X6 = col_double(),
  X7 = col_double(),
  X8 = col_double(),
  X9 = col_double(),
  X10 = col_double()
)
Parsed with column specification:
cols(
  X1 = col_double()
)


### check if Y is scaled
calculate the mean and sd of each column, it should be zero center and `sd == 1`

In [20]:
apply(Y, 2, function(x){
    res        = c(mean(x), sd(x))
    names(res) = c("Mean", "SD")
    return(res)
})

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10
Mean,3.392408e-16,-1.008253e-16,-2.208771e-17,-3.006082e-17,8.736955e-17,2.123577e-16,5.108118e-18,-9.419387e-17,-1.34685e-16,6.933805e-17
SD,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.00000e+00,1.000000e+00


### check if C is correctly setup

In [21]:
table(C)

C
    1     2     3     4     5     6     7     8     9    10    11    12    13 
10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 
   14    15    16    17    18 
10000 10000 10000 10000 10000 

# Code for running MPSK

Original code from Cliburn
```
set.seed(1)
Y <- scale(Y)
res <- mpsk(Y, C)
```

conversation between Cliburn and Shai

**Cliburn's question**

<font color = "blue">
Hi Shai,

I have another question - when we fit GMMs to similar data, we would often see in excess of 50 clusters by DP methods, yet we only see 9 clusters when Kuei fitted the 8-color ICS data. Part of this is probably simply that the skew distributions and allowing perturbed locations results in a more parsimonious model, but it is also likely that small cluster are not being captured separately. Should we be concerned about this, and if so, are there prior or other parameters we can set to increase the likelihood of “splitting” clusters?

Thanks,

Cliburn
</font>

**Shai's reply**

<font color = "red">
Hi Cliburn,
    
The low number of clusters is probably a result of the maximal number of clusters for the finite approximation, which is by default 10. To set a higher cap:
</font>

```   
prior=list()
prior$K = 50
res = mpsk(Y, C, prior=prior) 
```

<font color = "red">
Of course, it may split clusters that need not be split, but hopefully it will do so less than Gaussian kernels.
Shai
</font>


In [22]:
set.seed(1)

prior=list()
prior$K = 50
res = mpsk(Y, C, prior=prior)

Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 9000000)”

In [23]:
class(res)

[1] "MPSK"

In [24]:
names(res)

[1] "chain" "data"  "prior" "pmc"

In [26]:
length(res$chain)

[1] 16

In [27]:
length(res$data)

[1] 2

In [28]:
length(res$prior)

[1] 10

In [30]:
length(res$pmc)

[1] 5

In [31]:
saveRDS(res, file = file.path(dat_dir, "ep8cs_mpsk_raw_prior50.RDS"))

In [33]:
tmp = system(paste("ls -l", dat_dir), intern = TRUE)
for(x in tmp){print(x)}

[1] "total 8154192"
[1] "-rw-r--r-- 1 clintko clintko    4500000 Dec  3 02:14 C.txt"
[1] "-rw-r--r-- 1 clintko clintko   36073280 Dec 17 00:36 ep8cs_dat_gaussnorm.txt"
[1] "-rw-r--r-- 1 clintko clintko   36444357 Dec 17 00:36 ep8cs_dat_mpsk_cal.txt"
[1] "-rw-r--r-- 1 clintko clintko   36433800 Dec 17 00:35 ep8cs_dat_mpsk_raw.txt"
[1] "-rw-r--r-- 1 clintko clintko 6639004546 Dec 16 01:58 ep8cs_mpsk_cal.RDS"
[1] "-rw-r--r-- 1 clintko clintko     179265 Dec 16 01:58 ep8cs_mpsk_chainSummary.RDS"
[1] "-rw-r--r-- 1 clintko clintko  762463121 Dec 28 05:13 ep8cs_mpsk_raw_prior50.RDS"
[1] "-rw-r--r-- 1 clintko clintko  724326313 Dec 16 01:53 ep8cs_mpsk_raw.RDS"
[1] "-rw-r--r-- 1 clintko clintko   35807668 Dec 16 01:53 ep8cs_mpsk_relab.RDS"
[1] "-rw-r--r-- 1 clintko clintko    9177084 Dec 17 05:29 ep8cs_tsne_gaussnorm.txt"
[1] "-rw-r--r-- 1 clintko clintko    1062006 Dec 17 01:49 ep8cs_tsne_idx.txt"
[1] "-rw-r--r-- 1 clintko clintko    9174880 Dec 17 05:17 ep8cs_tsne_mpsk_cal.txt"
[1] "-rw-r--r-